# ToDo
***Preprocessing:***
- clean :
    - rows with NaN?
    - rows with inly punctuation?
- ~~word embeddings from fasttext~~
- ~~mention sequences~~
- phi_1 : ~~embeddings of first 3 words in m~~
- phi_2 : 
    - ~~embeddings of 3 proceeding words~~
    - ~~embeddings of 3 succeeding words~~
    - ~~avg embedings of all words in m~~
- phi_3 : 
    - ~~Embeddings of 3 proceeding sentences~~
    - ~~Embeddings of 1 succeeding sentence~~
    - ~~Embedding of the current sentence~~
- phi_4 : 
    - ~~Embeddings of 3 proceeding utterances~~
    - ~~Embeddings of 1 succeeding utterances~~
    - ~~Embeddings of the current utterance~~
- ~~embeddings~~
- put everything in np arrays
- get_features function
- main function
- episode and scene differences

In [242]:
import pandas as pd
import numpy as np
import fasttext

In [243]:
def __main__():
    ## hyperparameters ##
    embeddings_size = 50
    text_transcript = 'data/text_transcript.txt'
    ## main ##
    train = pd.read_csv('data/friends.train.episode_delim.conll',sep='\s+',header=None,comment='#')
    train_scene = pd.read_csv('data/friends.train.scene_delim.conll',sep='\s+',header=None,comment='#')
    trial = pd.read_csv('data/friends.trial.episode_delim.conll',sep='\s+',header=None,comment='#')
    trial_scene = pd.read_csv('data/friends.trial.scene_delim.conll',sep='\s+',header=None,comment='#')
    dfs = [train,train_scene,trial,trial_scene]
    dfs = [clean(df) for df in dfs]
    with open(text_transcript,'wt') as f:
        f.write(make_transcript(dfs))
    embeddings_model = fasttext.skipgram(text_transcript,'embeddings_model',min_count=1,dim=50)
    feature_matrices = []
    for df in dfs:
        feature_matrices.append(make_feature_matrices(train))

In [244]:
def make_transcript(dfs,idx=6):
    words = ''
    for df in dfs:
        words += ' '.join(list(df[idx]))
    return words

In [245]:
def clean(df):
    df = df[[0,2,3,4,5,6,9,10,11]]
    return df

In [246]:
def get_mention_rows(df):
    return df[df[11]!='-'][df[11]!='NaN'][df[11]!=np.nan]

In [247]:
def get_words(df):
    l = df[6].values
    return l, range(len(l))

In [274]:
def get_pre_words(mentions_idx,words,words_idx):
    pre_words = []
    for idx in mentions_idx:
        pre_words.append([\
                           words[words_idx[idx[0]-1]] if (idx[-1]-1)>=0 else '',\
                           words[words_idx[idx[0]-2]] if (idx[-1]-2)>=0 else '',\
                           words[words_idx[idx[0]-3]] if (idx[-1]-3)>=0 else ''\
                          ])
    return pre_words

In [280]:
def get_next_words(mentions_idx,words,words_idx):
    next_words = []
    for idx in mentions_idx:
        next_words.append([\
                           words[words_idx[idx[0]+1]] if (idx[-1]+1)<len(words) else '',\
                           words[words_idx[idx[0]+2]] if (idx[-1]+2)<len(words) else '',\
                           words[words_idx[idx[0]+3]] if (idx[-1]+3)<len(words) else ''\
                          ])
    return next_words

In [250]:
def get_sent_idx(df):
    sents = []
    sents_idx = []
    cnt = 0
    for row_no,row in df.iterrows():
        if row[2]==0:
            if row_no!=0:
                cnt +=1
                sents.append(sent_buf)
            sent_buf = row[6]
        else:
            sent_buf += ' ' + row[6]
        sents_idx.append(cnt)
    if df.iloc[len(df)-1][2]!=0:
        sents.append(sent_buf)
        sents_idx.append(cnt)
    return sents,sents_idx

In [251]:
def get_utterances_idx(df):
    ut = []
    ut_idx = []
    cnt = 0
    speaker = None
    for row_no,row in df.iterrows():
        if row[9]!=speaker:
            if row_no!=0:
                ut.append(ut_buf)
                cnt += 1
            speaker = row[9]
            ut_buf = row[6]
        else:
            ut_buf += ' ' + row[6]
        ut_idx.append(cnt)
    if df.iloc[len(df)-1][9]==speaker:
        ut.append(ut_buf)
        ut_idx.append(cnt)
    return ut,ut_idx

In [252]:
def get_mention_arrays(df):
    mentions = []
    mentions_y = []
    mentions_idx = []
    mention_f = False
    for row_no,row in df.iterrows():
        if mention_f and row[11]=='-':
            mention_buf += ' ' + row[6]
        elif row[11]=='-' or type(row[11])==float:
            continue
        elif ('(' in row[11]) and (')' in row[11]):
            mentions.append(row[6])
            mentions_y.append(int(row[11][1:-1]))
            mentions_idx.append([row_no,row_no])
        elif ('(' in row[11]):
            mention_f = True
            mention_buf = row[6]
            mention_idx_buf = row_no
        elif (')' in row[11]):
            mention_f = False
            mention_buf += ' ' + row[6]
            mentions.append(mention_buf)
            mentions_y.append(int(row[11][:-1]))
            mentions_idx.append([mention_idx_buf,row_no])
    return mentions,mentions_y,mentions_idx

In [253]:
def get_current_sents(mentions_idx,sents,sents_idx):
    curr_sents = []
    for idx in mentions_idx:
        curr_sents.append(sents[sents_idx[idx[0]]])
    return curr_sents

In [254]:
def get_next_sents(mentions_idx,sents,sents_idx):
    next_sents = []
    for idx in mentions_idx:
        t = sents_idx[idx[-1]]+1
        if t<len(sents):
            next_sents.append(sents[t])
    return next_sents

In [255]:
def get_pre_sents(mentions_idx,sents,sents_idx):
    pre_sents = []
    for idx in mentions_idx:
        t = sents_idx[idx[0]]
        pre_sents.append([\
                         sents[t-1] if (t-1)>=0 else '',\
                          sents[t-2] if (t-2)>=0 else '',\
                          sents[t-3] if (t-3)>=0 else ''\
                         ])
    return pre_sents

In [256]:
def get_current_utterance(mentions_idx,ut,ut_idx):
    curr_ut = []
    for idx in mentions_idx:
        curr_ut.append(ut[ut_idx[idx[0]]])
    return curr_ut

In [257]:
def get_next_utterances(mentions_idx,ut,ut_idx):
    next_ut = []
    for idx in mentions_idx:
        t = ut_idx[idx[-1]]+1
        if t<len(ut):
            next_ut.append(ut[t])
    return next_ut

In [258]:
def get_pre_utterances(mentions_idx,ut,ut_idx):
    pre_ut = []
    for idx in mentions_idx:
        t = ut_idx[idx[0]]
        pre_ut.append([\
                         ut[t-1] if (t-1)>=0 else '',\
                          ut[t-2] if (t-2)>=0 else '',\
                          ut[t-3] if (t-3)>=0 else ''\
                         ])
    return pre_ut

In [259]:
def get_embeddings(l):
    return np.array([embeddings_model[i] for i in l])

In [320]:
def get_avg_embeddings(l):
    a = [np.array([embeddings_model[i] for i in sent.split()]) for sent in l]
    return np.array([np.sum(i,axis=0)/len(i) for i in a])

In [414]:
def get_multiple_avg_embeddings(l):
    ret = []
    for sent_arr in l:
        sent_arr_emb = []
        for sent in sent_arr:
            if sent=='':
                sent_arr_emb.append([0]*embeddings_size)
            else:
                sent_emb = []
                for word in sent.split():
                    sent_emb.append(embeddings_model[word])
                sent_arr_emb.append(np.sum(sent_emb,axis=0)/len(sent_emb))
        ret.append(sent_arr_emb)
    return np.array(ret)

In [261]:
train = pd.read_csv('data/friends.train.episode_delim.conll',sep='\s+',header=None,comment='#')
df = clean(train)

In [262]:
mentions,mentions_y,mentions_idx = get_mention_arrays(df)

In [263]:
words,words_idx = get_words(df)

In [277]:
pre_words = get_pre_words(mentions_idx,words,words_idx)

In [284]:
next_words = get_next_words(mentions_idx,words,words_idx)

In [266]:
sents,sents_idx = get_sent_idx(df)

In [267]:
curr_sents = get_current_sents(mentions_idx,sents,sents_idx)

In [268]:
next_sents = get_next_sents(mentions_idx,sents,sents_idx)

In [269]:
pre_sents = get_pre_sents(mentions_idx,sents,sents_idx)

In [270]:
ut,ut_idx = get_utterances_idx(df)

In [271]:
curr_ut = get_current_utterance(mentions_idx,ut,ut_idx)

In [272]:
next_ut = get_next_utterances(mentions_idx,ut,ut_idx)

In [273]:
pre_ut = get_pre_utterances(mentions_idx,ut,ut_idx)

In [451]:
def get_phi_1(mentions):
    phi = []
    for i in mentions:
        t = i.split()
        if len(t)==0:
            raise ValueError('Empty mention')
        elif len(t)==1:
            phi.append(get_embeddings(t+['','']))
        elif len(t)==2:
            phi.append(get_embeddings(t+['']))
        else:
            phi.append(get_embeddings(t[0:3]))
    return np.array(phi)

def get_phi_2(pre_words,next_words,mentions):
    pre = [get_embeddings(i) for i in pre_words]
    suc = [get_embeddings(i) for i in next_words]
    avg = get_avg_embeddings(mentions)
    return np.array([np.vstack((i[0],i[1],i[2],j[0],j[1],j[2],k)) for i,j,k in zip(pre,suc,avg)])

def get_phi_3(pre_sents,next_sents,curr_sents):
    p = get_multiple_avg_embeddings(pre_sents)
    s = get_avg_embeddings(next_sents)
    c = get_avg_embeddings(curr_sents)
    return np.array([np.vstack((i[0],i[1],i[2],j,k)) for i,j,k in zip(p,s,c)])

def get_phi_4(pre_ut,next_ut,curr_ut):
    p = get_multiple_avg_embeddings(pre_ut)
    s = get_avg_embeddings(next_ut)
    c = get_avg_embeddings(curr_ut)
    return np.array([np.vstack((i[0],i[1],i[2],j,k)) for i,j,k in zip(p,s,c)])

In [457]:
phi = [0]*5
phi[1] = get_phi_1(mentions)
phi[2] = get_phi_2(pre_words,next_words,mentions)
phi[3] = get_phi_3(pre_sents,next_sents,curr_sents)
phi[4] = get_phi_4(pre_ut,next_ut,curr_ut)
phi[1] = np.reshape(phi[1],[13280,3,50,1])
phi[2] = np.reshape(phi[2],[13280,7,50,1])
phi[3] = np.reshape(phi[3],[13280,5,50,1])
phi[4] = np.reshape(phi[4],[13280,5,50,1])

In [473]:
def make_mention_pairs(phi,mentions_y,window=7):
    tuples = []
    for i in range(len(phi[1])):
        for j in range(i+1,window+i+1):
            if j<len(phi[1]):
                tuples.append([\
                              [phi[k][i] for k in range(1,5)],\
                              [phi[k][j] for k in range(1,5)],\
                              1 if mentions_y[i]==mentions_y[j] else 0\
                             ])
            else:
                break
    return tuples

In [474]:
pairs = make_mention_pairs(phi,mentions_y)

In [475]:
len(pairs)

92932

In [477]:
13280*7

92960

In [240]:
import fasttext
model = fasttext.skipgram('data/001.txt','model',min_count=1,dim=50)

In [234]:
embeddings_model = model

In [ ]:
with open('data/001.txt','rt') as f:
    l = f.read().split('\n\n')
    a = get_avg_embeddings(l)
    print(len(l),len(a))
    print(a)

In [179]:
pd.options.display.max_rows = 5

In [180]:
df

,0,2,3,4,5,6,9,10,11
0,/friends-s01e01,0,There,EX,(TOP(S(NP*),there,Monica_Geller,*,-
1,/friends-s01e01,1,'s,VBZ,(VP*,be,Monica_Geller,*,-
...,...,...,...,...,...,...,...,...,...
160169,/friends-s02e24,5,girl,NN,*)),girl,Joey_Tribbiani,*,-
160170,/friends-s02e24,6,.,.,*)),.,Joey_Tribbiani,*,-


In [36]:
with open('remove_me.txt','wt') as f:
    print(df,file=f)